# Step 2

2018 10 25

optimization of the bi ADMM

focused on vectorlization

more detailed file

# Summary time diff

#### L_num

In [116]:
t1 = Sys.time()
x1 = L_num_old(100)
t2 = Sys.time()
t2-t1

Time difference of 0.09051704 secs

In [117]:
t3 = Sys.time()
x2 = L_num(100)
t4 = Sys.time()
t4-t3

Time difference of 0.005729198 secs

#### update lambda

In [114]:
t1 = Sys.time()
res1 = update_lambda_old(X, A, nu1, nu2,v, z)
t2 = Sys.time()
t2-t1

Time difference of 0.2200952 secs

In [115]:
t3 = Sys.time()
res2 = update_lambda(X, A, nu1, nu2,v, z)
t4 = Sys.time()
t4-t3

Time difference of 0.0204699 secs

#### update vz

In [178]:
t1 = Sys.time()
res1 = update_vz_old(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2)
t2 = Sys.time()
t2-t1

Time difference of 0.1449609 secs

In [179]:
t3 = Sys.time()
res2 = update_vz(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2,n,p,eplison_n,eplison_p)
t4 = Sys.time()
t4-t3

Time difference of 0.02553797 secs

#### sylvester equation

In [196]:
A = matrix(rnorm(3600,0,6),60,60)
B = matrix(rnorm(360*360,0,6),360,360)
X = matrix(rnorm(360*60,0,7),60,360)
C = A%*% X + X%*% B
XX = X

In [125]:
t1 =Sys.time()
res = sylvester_old(A,B,C)
t2 =Sys.time()
t2-t1

Time difference of 2.750152 secs

In [126]:
t3 =Sys.time()
res = sylvester(A,B,C)
t4 =Sys.time()
t4-t3

Time difference of 0.3254192 secs

In [251]:
t1 = Sys.time()
res1 = update_A_old(X, nu1, nu2, lambda_1, lambda_2, v, z)
t2 = Sys.time()
t2-t1

Time difference of 2.156838 mins

In [253]:
t3 = Sys.time()
res2 = update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p)
t4 = Sys.time()
t4-t3

Time difference of 9.107001 secs

### 1. L_num function

old

In [1]:
L_num_old=function(n){
  library(tidyr)
  L=matrix(0,n,n)
  for(i in 1:n){
    for(j in 1:n){
      L[i,j] = paste(i,',',j)
    }
  }
  l=c(L[upper.tri(L)])
  ll=data.frame(l)
  
  LL=ll %>% separate(l,c('l1','l2'),sep=',')
  LL=data.frame(l1=as.numeric(LL[,'l1']),l2=as.numeric(LL[,'l2']))
  return(LL)
}

new

In [82]:
L_num = function(n){
  t = matrix(0, n*(n-1)/2,2)
  count = 0
  for(i in 1:(n-1)){
    for(j in (i+1):n){
      count = count + 1
      t[count,1]= i; t[count,2]=j
    }
  }
  LL = data.frame(l1 = t[,1],l2 = t[,2])
  LL = LL[order(LL$l2),]
  rownames(LL) = NULL
  return(LL)
}

In [83]:
L_num_old(5)

l1,l2
1,2
1,3
2,3
1,4
2,4
3,4
1,5
2,5
3,5
4,5


In [84]:
L_num(5)

l1,l2
1,2
1,3
2,3
1,4
2,4
3,4
1,5
2,5
3,5
4,5


test

In [85]:
t1 = Sys.time()
x1 = L_num_old(100)
t2 = Sys.time()
t2-t1

Time difference of 0.117928 secs

In [86]:
t3 = Sys.time()
x2 = L_num(100)
t4 = Sys.time()
t4-t3

Time difference of 0.006487846 secs

### 2. Sylester equation

In [22]:
A = matrix(rnorm(3600,0,6),60,60)
B = matrix(rnorm(360*360,0,6),360,360)
X = matrix(rnorm(360*60,0,7),60,360)
C = A%*% X + X%*% B
XX = X

In [23]:
library(Matrix)
A1 = Schur(A)
Q1 = A1$Q; R1 = A1$T

A2 = Schur(B)
Q2 = A2$Q; R2 = A2$T 
C = t(Q1) %*% C %*% Q2

Rsq = R1 * R1
I = diag(dim(A)[1])

k = 1
n = dim(R2)[1]

#### part 1:

In [24]:
t1 = Sys.time()
for(k in 1:n){
  temp = matrix(0, dim(X)[1],1)
  if(k == 1){
    temp = temp
  }else{
    for(i in 1:(k-1)){
      temp = temp + X[,i] * R2[i,k]
    }
  }
  temp = matrix(temp, dim(C)[1],1)
}
t2 = Sys.time()
t2-t1

Time difference of 0.132673 secs

In [26]:
t3 = Sys.time()
for(k in 1:(n)){
  temp = matrix(0, dim(X)[1],1)
  if(k == 1){
    temp = temp
  }else{
    temp = (X[,1:(k-1)]) %*% matrix(R2[1:(k-1),k],k-1,1)
  }
}
t4 = Sys.time()
t4-t3

Time difference of 0.06556201 secs

#### part 2

In [30]:
t1 = Sys.time()
for(k in 1:(n-1)){
  r11 = R2[k,k]; r12 = R2[k, k+1]; r21 = R2[k+1, k]; r22 = R2[k+1, k+1]
  temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
  if(k == 1){
    temp2 = temp2
    temp3 = temp3
  }else{
    for(i in 1:(k-1)){
      temp2 = temp2 + X[,i] * R2[i,k]
      temp3 = temp3 + X[,i] * R2[i,k+1]}
  }
  temp2 = matrix(temp2, dim(X)[1],1)
  temp3 = matrix(temp3, dim(X)[1],1)
}

t2 = Sys.time()
t2-t1

Time difference of 0.3974068 secs

In [31]:
t3 = Sys.time()
for(k in 1:(n-1)){
  r11 = R2[k,k]; r12 = R2[k, k+1]; r21 = R2[k+1, k]; r22 = R2[k+1, k+1]
  temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
  if(k == 1){
    temp2 = temp2
    temp3 = temp3
  }else{
    temps = X[,1:(k-1)] %*% matrix(R2[1:(k-1),k:(k+1)],k-1,2)
    temp2 = temps[,1]
    temp3 = temps[,2]
  }
}
t4 = Sys.time()
t4-t3

Time difference of 0.07258606 secs

old equation

In [32]:
sylvester_old = function(A, B, C, eps = 0.0001){
  
  library(MASS)
  library(Matrix)
  
  A1 = Schur(A)
  Q1 = A1$Q; R1 = A1$T
  
  A2 = Schur(B)
  Q2 = A2$Q; R2 = A2$T 
  C = t(Q1) %*% C %*% Q2
  
  Rsq = R1 * R1
  I = diag(dim(A)[1])
  
  k = 1
  n = dim(R2)[1]
  
  while(k < n + 1){
    if(k < n){
      if(abs(R2[k+1, k]) < eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          for(i in 1:(k-1)){
            temp = temp + X[,i] * R2[i,k]
          }
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
       # mytry = myTryCatch(solve(left))
        #if(is.null(mytry$error) == 0){er = c(er,tt)}
        k = k+1
      }else{
        r11 = R2[k,k]; r12 = R2[k, k+1]; r21 = R2[k+1, k]; r22 = R2[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        if(k == 1){
          temp2 = temp2
          temp3 = temp3
        }else{
          for(i in 1:(k-1)){
            temp2 = temp2 + X[,i] * R2[i,k]
            temp3 = temp3 + X[,i] * R2[i,k+1]}
        }
        temp2 = matrix(temp2, dim(X)[1],1)
        temp3 = matrix(temp3, dim(X)[1],1)
        b1 = C[,k] - temp2 
        b2 = C[,k+1] - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        X[,k:(k+1)] = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                             (r11*r22 - r12*r21) * I) %*% b_prime 
        k = k+2
      }
    }else{
      if(abs(R2[1, k]) > eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          for(i in 1:(k-1)){
            temp = temp + X[,i] * R2[i,k]
          }
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
        k = k+1
      }else{
        R22 = R2
        R22 = cbind(R2, rep(0,dim(R2)[1]))
        R22 = rbind(R22,rep(0,dim(R2)[1]+1))
        r11 = R22[k,k]; r12 = R22[k, k+1]; r21 = R22[k+1, k]; r22 = R22[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        for(i in 1:(k-1)){
          temp2 = temp2 + X[,i] * R22[i,k]
          temp3 = temp3 + X[,i] * R22[i,k+1]}
        temp2 = matrix(temp2, dim(X)[1],1)
        temp3 = matrix(temp3, dim(X)[1],1)
        b1 = C[,k] - temp2 
        b2 = - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        GOD = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                     (r11*r22 - r12*r21) * I) %*% b_prime 
        X[,k] = GOD[,1]
        k = k+2
      }
    }
  }
  return(Q1 %*% X %*% t(Q2))
}

In [221]:
sylvester= function(A, B, C, eps = 0.0001){
  
  library(MASS)
  library(Matrix)
  
  A1 = Schur(A)
  Q1 = A1$Q; R1 = A1$T
  
  A2 = Schur(B)
  Q2 = A2$Q; R2 = A2$T 
  C = t(Q1) %*% C %*% Q2
  
  Rsq = R1 * R1
  I = diag(dim(A)[1])
  
  k = 1
  n = dim(R2)[1]
  
  while(k < n + 1){
    if(k < n){
      if(abs(R2[k+1, k]) < eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          temp = (X[,1:(k-1)]) %*% matrix(R2[1:(k-1),k],k-1,1)
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
       # mytry = myTryCatch(solve(left))
        #if(is.null(mytry$error) == 0){er = c(er,tt)}
        k = k+1
      }else{
        r11 = R2[k,k]; r12 = R2[k, k+1]; r21 = R2[k+1, k]; r22 = R2[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        if(k == 1){
          temp2 = temp2
          temp3 = temp3
        }else{
          temps = X[,1:(k-1)] %*% matrix(R2[1:(k-1),k:(k+1)],k-1,2)
            temp2 = temps[,1]
            temp3 = temps[,2]
        }
        b1 = C[,k] - temp2 
        b2 = C[,k+1] - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        X[,k:(k+1)] = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                             (r11*r22 - r12*r21) * I) %*% b_prime 
        k = k+2
      }
    }else{
      if(abs(R2[1, k]) > eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          temp = (X[,1:(k-1)]) %*% matrix(R2[1:(k-1),k],k-1,1)
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
        k = k+1
      }else{
        R22 = R2
        R22 = cbind(R2, rep(0,dim(R2)[1]))
        R22 = rbind(R22,rep(0,dim(R2)[1]+1))
        r11 = R22[k,k]; r12 = R22[k, k+1]; r21 = R22[k+1, k]; r22 = R22[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        
        if(k == 1){
          temp2 = temp2
          temp3 = temp3
        }else{
          temps = X[,1:(k-1)] %*% matrix(R22[1:(k-1),k:(k+1)],k-1,2)
          temp2 = temps[,1]
          temp3 = temps[,2]
        }
          
        b1 = C[,k] - temp2 
        b2 = - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        GOD = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                     (r11*r22 - r12*r21) * I) %*% b_prime 
        X[,k] = GOD[,1]
        k = k+2
      }
    }
  }
  return(Q1 %*% X %*% t(Q2))
}

new equation

In [218]:
sylvester_o(M,t(N),C)

0.2306219,-0.6412549,0.2306219
1.0185384,4.0862442,1.0185384
0.2770504,-0.3626837,0.2770504
0.4987754,0.9676665,0.4987754


In [220]:
sylvester_old(M,t(N),C)

0.2306219,-0.6412549,0.2306219
1.0185384,4.0862442,1.0185384
0.2770504,-0.3626837,0.2770504
0.4987754,0.9676665,0.4987754


In [34]:
t1 =Sys.time()
res = sylvester_old(A,B,C)
t2 =Sys.time()
t2-t1

Time difference of 1.953639 secs

In [35]:
t3 =Sys.time()
res = sylvester(A,B,C)
t4 =Sys.time()
t4-t3

Time difference of 0.5704648 secs

### 3. update A

In [232]:
alk = function(A,n,p){
  count = 0
  el1 = el2 = matrix(0,n,n*(n-1)/2)
  for(i in 1:(n-1)){
    el1[,(count+1):(count+i)] = diag(1,n,i)
    temp = matrix(0,n,i)
    temp[i,] = 1
    el2[,(count+1):(count+i)] = temp
    count = count+i
  }
  el2 = rbind(el2[n,],el2[1:(n-1),])
  al1 = t(A) %*% el1; al2 = t(A) %*% el2
  
  # k
  count = 0
  ek1 = ek2 = matrix(0,p,p*(p-1)/2)
  for(i in 1:(p-1)){
    ek1[,(count+1):(count+i)] = diag(1,p,i)
    temp = matrix(0,p,i)
    temp[i+1,] = 1
    ek2[,(count+1):(count+i)] = temp
    count = count+i
  }
  ak1 = A %*% ek1; ak2 = A %*% ek2
  return(list(al1 = al1, al2 = al2, ak1 = ak1, ak2 = ak2,
              el1 = el1, el2 = el2, ek1 = ek1, ek2 = ek2))
}

In [240]:
X = matrix(1:12,4,3)
n=dim(X)[1]; p=dim(X)[2]
eplison_p=L_num(p)
eplison_n=L_num(n)
nu1=1; nu2=1
lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

library(MASS)
library(Matrix)
A= X
n=dim(X)[1]; p=dim(X)[2]

In [246]:
update_A_old = function(X, nu1, nu2, lambda_1, lambda_2, v, z){
  library(MASS)
  library(Matrix)
  n=dim(X)[1]; p=dim(X)[2]
  eplison_n=L_num(n)
  eplison_p=L_num(p)
  
  En=matrix(0,n,n)
  Ep=matrix(0,p,p)
  
  # calculate \sum_{l \in \epsilon_1}(e_{l1} - e_{l_2})(e_{l1} - e_{l_2})^T, which is En
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    el1=matrix(rep(0,n),n,1)
    el2=matrix(rep(0,n),n,1)
    el1[l1,1]=1;el2[l2,1]=1
    En=En+(el1-el2) %*% t(el1-el2)
  }
  
  # calculate \sum_{k \in \epsilon_2} (e_{k1} - e_{k_2})(e_{k1} - e_{k_2})^T, which is Ep
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[i,'l1']
    l2=eplison_p[i,'l2']
    el1=matrix(rep(0,p),p,1)
    el2=matrix(rep(0,p),p,1)
    el1[l1,1]=1;el2[l2,1]=1
    Ep=Ep+(el1-el2) %*% t(el1-el2)
  }
  
  M = diag(1,n,n) + nu1 * En
  
  N = nu2 * Ep
  t3 = Sys.time()
  # C2 is the second part of C, which is \sum_{l \in \epsilon_1} (e_{l1} - e_{l_2})(\lambda_{l1} + \nu_1 v_l)^T 
  C2 = matrix(0,n,p) 
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    el1=matrix(rep(0,n),n,1)
    el2=matrix(rep(0,n),n,1)
    el1[l1,1]=1;el2[l2,1]=1
    C2 = C2 + (el1-el2) %*% t(lambda_1[,i]+ nu1 * v[,i])
  }
  t4= Sys.time()
  # C3 is the third part of C, which is \sum_{k \in \epsilon_2} (\lambda_{k2} + \nu_2 z_k)(e_{k1} - e_{k_2})^T
  C3 = matrix(0,n,p)
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[i,'l1']
    l2=eplison_p[i,'l2']
    el1=matrix(rep(0,p),p,1)
    el2=matrix(rep(0,p),p,1)
    el1[l1,1]=1;el2[l2,1]=1
    C3 = C3+(lambda_2[,i] + nu2 * z[,i]) %*% t(el1-el2)
  }
  
  C = X +  C2 + C3  
  
  A = sylvester(M,t(N),C)
  
  return(A)
}

In [247]:
update_A = function(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p){
  En = diag(0:(n - 1)) + diag((n - 1):0) - matrix(1, n, n) + diag(1, n, n)
  Ep = diag(0:(p - 1)) + diag((p - 1):0) - matrix(1, p, p) + diag(1, p, p)
  
  M = diag(1,n,n) + nu1 * En
  
  N = nu2 * Ep
  
  alk = alk(X,n,p)
  
  el1=alk$el1
  el2=alk$el2
  ek1=alk$ek1
  ek2=alk$ek2
  
  lv = lambda_1+ nu1 * v
  lz = lambda_2 + nu2 * z
  C2 = 0 -(el2-el1) %*% t(lv)
  C3 = lz %*% t(ek1-ek2)
  C = X +  C2 + C3  
  
  A = sylvester(M,t(N),C)
  return(A)
}

In [248]:
update_A_old(X, nu1, nu2, lambda_1, lambda_2, v, z)

7.4,7.4,7.4
6.8,6.8,6.8
6.2,6.2,6.2
5.6,5.6,5.6


In [249]:
update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p)

7.4,7.4,7.4
6.8,6.8,6.8
6.2,6.2,6.2
5.6,5.6,5.6


check time

In [250]:
X = matrix(rnorm(40000),400,100)
n=dim(X)[1]; p=dim(X)[2]
eplison_p=L_num(p)
eplison_n=L_num(n)
nu1=1; nu2=1
lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

library(MASS)
library(Matrix)
A= X
n=dim(X)[1]; p=dim(X)[2]

In [251]:
t1 = Sys.time()
res1 = update_A_old(X, nu1, nu2, lambda_1, lambda_2, v, z)
t2 = Sys.time()
t2-t1

Time difference of 2.156838 mins

In [252]:
t3 = Sys.time()
res2 = update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p)
t4 = Sys.time()
t4-t3

Time difference of 9.546415 secs

### 4. update v z 

In [180]:
gamma_1 = 0.5; gamma_2 = 0

nu1 = 1; nu2 = 1; 
X = matrix(1:30,6,5)
A = X
n = dim(X)[1]; p = dim(X)[2]
eplison_p = L_num(p)
eplison_n = L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

In [153]:
dist_weight <- function(X, phi){
  dist_X <- as.numeric(dist(X))
  return(exp(- phi * dist_X^2))
}
knn_weights <- function(w,k,n) {
  i <- 1
  neighbors <- tri2vec(i,(i+1):n,n)
  keep <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
  for (i in 2:(n-1)) {
    group_A <- tri2vec(i,(i+1):n,n)
    group_B <- tri2vec(1:(i-1),i,n)
    neighbors <- c(group_A,group_B)
    knn <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
    keep <- union(knn,keep)
  }
  i <- n
  neighbors <- tri2vec(1:(i-1),i,n)
  knn <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
  keep <- union(knn,keep)
  w[-keep] <- 0
  return(w)
}

tri2vec <- function(i,j,n) {
  return(n*(i-1) - i*(i-1)/2 + j -i)
}

prox = function(v,sigma){
  return(max(1-sigma/sum(t(v) %*% v),0) * v)
}

old

In [164]:
update_vz_old = function(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2){
  
  n=dim(X)[1]; p=dim(X)[2]
  eplison_n=L_num(n)
  eplison_p=L_num(p)
 
  w <- dist_weight(X / sqrt(p),phi = 0.5)
  w_l <- knn_weights(w,k = 1,n)
  
  u = dist_weight(t(X) / sqrt(n),phi = 0.5)
  u_k <- knn_weights(u,k = 1,p)
  
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    a_l1 = A[l1,]; a_l2 = A[l2,]
    v_temp = a_l1 - a_l2 - 1/nu1 * lambda_1[,i]
    sigma_1l = gamma_1 * w_l[i]/nu1
    v[,i] = prox(v_temp,sigma_1l)
  }
  
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[i,'l1']
    l2=eplison_p[i,'l2']
    a_l1 = A[,l1]; a_l2 = A[,l2]
    v_temp = a_l1 - a_l2 - 1/nu2 * lambda_2[,i]
    #u_k = exp(-0.5 * (t(X[,l1] - X[,l2]) %*% (X[,l1] - X[,l2])))
    sigma_2k = gamma_2 * u_k[i]/nu2
    z[,i] = prox(v_temp,sigma_2k)
  }
  
  return(list(v = v, z = z))
}

In [165]:
update_vz_old(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2)

$v
          [,1] [,2] [,3] [,4] [,5]      [,6] [,7] [,8] [,9] [,10] [,11] [,12]
[1,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[2,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[3,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[4,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[5,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
         [,13] [,14]     [,15]
[1,] -3.984837    -3 -1.969673
[2,] -3.984837    -3 -1.969673
[3,] -3.984837    -3 -1.969673
[4,] -3.984837    -3 -1.969673
[5,] -3.984837    -3 -1.969673

$z
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[2,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[3,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[4,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[5,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[6,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7

To update v,z,lambda: calculate al1,al2,ak1,ak2 once may save time

$a_{l1}, a_{l2}$

In [171]:
alk = function(A,n,p){
  count = 0
  al1 = al2 = matrix(0,n,n*(n-1)/2)
  for(i in 1:(n-1)){
    al1[,(count+1):(count+i)] = diag(1,n,i)
    temp = matrix(0,n,i)
    temp[i,] = 1
    al2[,(count+1):(count+i)] = temp
    count = count+i
  }
  al2 = rbind(al2[n,],al2[1:(n-1),])
  al1 = t(A) %*% al1; al2 = t(A) %*% al2
  
  # k
  count = 0
  ak1 = ak2 = matrix(0,p,p*(p-1)/2)
  for(i in 1:(p-1)){
    ak1[,(count+1):(count+i)] = diag(1,p,i)
    temp = matrix(0,p,i)
    temp[i+1,] = 1
    ak2[,(count+1):(count+i)] = temp
    count = count+i
  }
  ak1 = A %*% ak1; ak2 = A %*% ak2
  return(list(al1 = al1, al2 = al2, ak1 = ak1, ak2 = ak2))
}

In [154]:
count = 0
al1 = al2 = matrix(0,n,n*(n-1)/2)
for(i in 1:(n-1)){
  al1[,(count+1):(count+i)] = diag(1,n,i)
  temp = matrix(0,n,i)
  temp[i,] = 1
  al2[,(count+1):(count+i)] = temp
  count = count+i
}
al2 = rbind(al2[n,],al2[1:(n-1),])
al1 = t(A) %*% al1; al2 = t(A) %*% al2

$a_{k1}, a_{k2}$

In [155]:
# k
count = 0
ak1 = ak2 = matrix(0,p,p*(p-1)/2)
for(i in 1:(p-1)){
  ak1[,(count+1):(count+i)] = diag(1,p,i)
  temp = matrix(0,p,i)
  temp[i+1,] = 1
  ak2[,(count+1):(count+i)] = temp
  count = count+i
}
ak1 = A %*% ak1; ak2 = A %*% ak2

In [156]:
n=dim(X)[1]; p=dim(X)[2]
eplison_n=L_num(n)
eplison_p=L_num(p)

w = dist_weight(X / sqrt(p),phi = 0.5)
w_l = knn_weights(w,k = 1,n)
u = dist_weight(t(X) / sqrt(n),phi = 0.5)
u_k = knn_weights(u,k = 1,p)

sigma_1 = gamma_1 * w_l/nu1
vtemp = al1 - al2 - 1/nu1 * lambda_1

temp1 = ifelse((1 - sigma_1/apply(vtemp^2,2,sum)) < 0, 0,1 - sigma_1/apply(vtemp^2,2,sum))
temp2 = matrix(temp1,dim(vtemp)[1],dim(vtemp)[2], byrow = TRUE) * vtemp 

v = temp2

ztemp = ak1 - ak2 - 1/nu2 * lambda_2
sigma_2 = gamma_2 * u_k/nu2

temp3 = ifelse((1 - sigma_2/apply(ztemp^2,2,sum)) < 0, 0,1 - sigma_2/apply(ztemp^2,2,sum))
temp4 = matrix(temp3,dim(ztemp)[1],dim(ztemp)[2], byrow = TRUE) * ztemp 

z = temp4

In [157]:
v

-1.969673,-3,-2,-4,-3,-1.969673,-5,-4,-3,-2,-6,-5,-3.984837,-3,-1.969673
-1.969673,-3,-2,-4,-3,-1.969673,-5,-4,-3,-2,-6,-5,-3.984837,-3,-1.969673
-1.969673,-3,-2,-4,-3,-1.969673,-5,-4,-3,-2,-6,-5,-3.984837,-3,-1.969673
-1.969673,-3,-2,-4,-3,-1.969673,-5,-4,-3,-2,-6,-5,-3.984837,-3,-1.969673
-1.969673,-3,-2,-4,-3,-1.969673,-5,-4,-3,-2,-6,-5,-3.984837,-3,-1.969673


In [158]:
z

-7,-13,-7,-19,-13,-7,-25,-19,-13,-7
-7,-13,-7,-19,-13,-7,-25,-19,-13,-7
-7,-13,-7,-19,-13,-7,-25,-19,-13,-7
-7,-13,-7,-19,-13,-7,-25,-19,-13,-7
-7,-13,-7,-19,-13,-7,-25,-19,-13,-7
-7,-13,-7,-19,-13,-7,-25,-19,-13,-7


new 

In [181]:
alk_value = alk(A,n,p)
al1 = alk_value$al1
al2 = alk_value$al2

ak1 = alk_value$ak1
ak2 = alk_value$ak2

In [182]:
update_vz = function(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2,n,p,eplison_n,eplison_p){
  w = dist_weight(X / sqrt(p),phi = 0.5)
  w_l = knn_weights(w,k = 1,n)
  u = dist_weight(t(X) / sqrt(n),phi = 0.5)
  u_k = knn_weights(u,k = 1,p)
  
    
  sigma_1 = gamma_1 * w_l/nu1
  vtemp = al1 - al2 - 1/nu1 * lambda_1
  
  temp1 = ifelse((1 - sigma_1/apply(vtemp^2,2,sum)) < 0, 0,1 - sigma_1/apply(vtemp^2,2,sum))
  temp2 = matrix(temp1,dim(vtemp)[1],dim(vtemp)[2], byrow = TRUE) * vtemp 
  
  v = temp2
  
  ztemp = ak1 - ak2 - 1/nu2 * lambda_2
  sigma_2 = gamma_2 * u_k/nu2
  
  temp3 = ifelse((1 - sigma_2/apply(ztemp^2,2,sum)) < 0, 0,1 - sigma_2/apply(ztemp^2,2,sum))
  temp4 = matrix(temp3,dim(ztemp)[1],dim(ztemp)[2], byrow = TRUE) * ztemp 
  
  z = temp4
  return(list(v=v,z=z))
}

In [183]:
n=dim(X)[1]; p=dim(X)[2]
eplison_n=L_num(n)
eplison_p=L_num(p)
update_vz(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2,n,p,eplison_n,eplison_p)

$v
          [,1] [,2] [,3] [,4] [,5]      [,6] [,7] [,8] [,9] [,10] [,11] [,12]
[1,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[2,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[3,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[4,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
[5,] -1.969673   -3   -2   -4   -3 -1.969673   -5   -4   -3    -2    -6    -5
         [,13] [,14]     [,15]
[1,] -3.984837    -3 -1.969673
[2,] -3.984837    -3 -1.969673
[3,] -3.984837    -3 -1.969673
[4,] -3.984837    -3 -1.969673
[5,] -3.984837    -3 -1.969673

$z
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[2,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[3,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[4,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[5,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7
[6,]   -7  -13   -7  -19  -13   -7  -25  -19  -13    -7

test time

In [168]:
gamma_1 = 0.5; gamma_2 = 0

nu1 = 1; nu2 = 1; 
X = matrix(1:3000,60,50)
A = X
n = dim(X)[1]; p = dim(X)[2]
eplison_p = L_num(p)
eplison_n = L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

In [178]:
t1 = Sys.time()
res1 = update_vz_old(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2)
t2 = Sys.time()
t2-t1

Time difference of 0.1449609 secs

In [179]:
t3 = Sys.time()
res2 = update_vz(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2,n,p,eplison_n,eplison_p)
t4 = Sys.time()
t4-t3

Time difference of 0.02553797 secs

### 5. update lambda

In [130]:
X = matrix(1:30,6,5)
A =  matrix(1:30,6,5)
n = dim(X)[1]; p = dim(X)[2]
eplison_p = L_num(p)
eplison_n = L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

nu1 = nu2 = 1

old

In [99]:
# update lambda 1
for(i in 1:dim(eplison_n)[1]){
  l1=eplison_n[i,'l1']
  l2=eplison_n[i,'l2']
  a_l1 = matrix(A[l1,],p,1)
  a_l2 = matrix(A[l2,],p,1)
  lambda_1[,i] = lambda_1[,i] + nu1 * (v[,i] - a_l1 + a_l2)
} 

# update lambda 2
for(i in 1:dim(eplison_p)[1]){
  l1=eplison_p[i,'l1']
  l2=eplison_p[i,'l2']
  a_k1 = matrix(A[,l1],n,1)
  a_k2 = matrix(A[,l2],n,1)
  lambda_2[,i] = lambda_2[,i] + nu2 * (z[,i] - a_k1 + a_k2)
}

In [100]:
lambda1= lambda_1;lambda2 = lambda_2

new

In [101]:
lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

In [102]:
count = 0
al1 = al2 = matrix(0,n,n*(n-1)/2)
for(i in 1:(n-1)){
  al1[,(count+1):(count+i)] = diag(1,n,i)
  temp = matrix(0,n,i)
  temp[i,] = 1
  al2[,(count+1):(count+i)] = temp
  count = count+i
}
al2 = rbind(al2[n,],al2[1:(n-1),])
lambda11 = lambda_1 + nu1 * (v - t(A) %*% al1 + t(A) %*% al2)

In [103]:
# k
count = 0
al1 = al2 = matrix(0,p,p*(p-1)/2)
for(i in 1:(p-1)){
  al1[,(count+1):(count+i)] = diag(1,p,i)
  temp = matrix(0,p,i)
  temp[i+1,] = 1
  al2[,(count+1):(count+i)] = temp
  count = count+i
}
lambda22 = lambda_2 + nu2 * (z - A %*% al1 + A %*% al2)

In [104]:
lambda1 == lambda11

TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE


In [105]:
lambda2 == lambda22

TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE


In [106]:
### update lambda
update_lambda_old=function(X, A, nu1, nu2,v, z){
  n = dim(X)[1]; p = dim(X)[2]
  eplison_p = L_num(p)
  eplison_n = L_num(n)
  # update lambda 1
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    a_l1 = matrix(A[l1,],p,1)
    a_l2 = matrix(A[l2,],p,1)
    lambda_1[,i] = lambda_1[,i] + nu1 * (v[,i] - a_l1 + a_l2)
  } 
  # update lambda 2
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[i,'l1']
    l2=eplison_p[i,'l2']
    a_k1 = matrix(A[,l1],n,1)
    a_k2 = matrix(A[,l2],n,1)
    lambda_2[,i] = lambda_2[,i] + nu2 * (z[,i] - a_k1 + a_k2)
  }
  return(lambda=list(lambda_1,lambda_2))
}

update_lambda=function(X, A, nu1, nu2,v, z){
  n = dim(X)[1]; p = dim(X)[2]
  eplison_p = L_num(p)
  eplison_n = L_num(n)
  # update lambda 1
  count = 0
  al1 = al2 = matrix(0,n,n*(n-1)/2)
  for(i in 1:(n-1)){
    al1[,(count+1):(count+i)] = diag(1,n,i)
    temp = matrix(0,n,i)
    temp[i,] = 1
    al2[,(count+1):(count+i)] = temp
    count = count+i
  }
  al2 = rbind(al2[n,],al2[1:(n-1),])
  lambda_1 = lambda_1 + nu1 * (v - t(A) %*% al1 + t(A) %*% al2)
  
  # update lambda 2
  count = 0
  al1 = al2 = matrix(0,p,p*(p-1)/2)
  for(i in 1:(p-1)){
    al1[,(count+1):(count+i)] = diag(1,p,i)
    temp = matrix(0,p,i)
    temp[i+1,] = 1
    al2[,(count+1):(count+i)] = temp
    count = count+i
  }
  lambda_2 = lambda_2 + nu2 * (z - A %*% al1 + A %*% al2)
  return(lambda=list(lambda_1,lambda_2))
}

In [107]:
update_lambda_old(X, A, nu1, nu2,v, z)

[[1]]
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[2,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[3,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[4,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[5,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
     [,15]
[1,]     3
[2,]     3
[3,]     3
[4,]     3
[5,]     3

[[2]]
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]    8   14    8   20   14    8   26   20   14     8
[2,]    8   14    8   20   14    8   26   20   14     8
[3,]    8   14    8   20   14    8   26   20   14     8
[4,]    8   14    8   20   14    8   26   20   14     8
[5,]    8   14    8   20   14    8   26   20   14     8
[6,]    8   14    8   20   14    8   26   20   14     8

In [108]:
update_lambda(X, A, nu1, nu2,v, z)

[[1]]
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[2,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[3,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[4,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
[5,]    3    4    3    5    4    3    6    5    4     3     7     6     5     4
     [,15]
[1,]     3
[2,]     3
[3,]     3
[4,]     3
[5,]     3

[[2]]
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]    8   14    8   20   14    8   26   20   14     8
[2,]    8   14    8   20   14    8   26   20   14     8
[3,]    8   14    8   20   14    8   26   20   14     8
[4,]    8   14    8   20   14    8   26   20   14     8
[5,]    8   14    8   20   14    8   26   20   14     8
[6,]    8   14    8   20   14    8   26   20   14     8

In [121]:
X = matrix(rnorm(3000),60,50)
A =  X
n = dim(X)[1]; p = dim(X)[2]
eplison_p = L_num(p)
eplison_n = L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

nu1 = nu2 = 1

In [112]:
t1 = Sys.time()
res1 = update_lambda_old(X, A, nu1, nu2,v, z)
t2 = Sys.time()
t2-t1

Time difference of 0.131958 secs

In [113]:
t3 = Sys.time()
res2 = update_lambda(X, A, nu1, nu2,v, z)
t4 = Sys.time()
t4-t3

Time difference of 0.012887 secs